In [1]:
%load_ext autotime

In [2]:
from etltools import s3
from lambda_client import (
    ClaimsClient,
    BenefitsClient,
    CalculatorClient,
    run_batch_on_schedule,
)
from lambda_client.config_info import ConfigInfo

aws_info = {
    'profile_name': 'sandbox',
}

time: 493 ms


In [3]:
uids = s3.read_json('s3://picwell.sandbox.medicare/samples/philadelphia-2015-1k-sample')

print '{} UIDs read'.format(len(uids))

1000 UIDs read
time: 207 ms


In [4]:
benefits_client = BenefitsClient(aws_info)
plans = benefits_client.get_by_state(['06'])  # CA
pids = [plan['picwell_id'] for plan in plans]

print '{} plans identified'.format(len(pids))

268 plans identified
time: 368 ms


In [5]:
configs = ConfigInfo('lambda_client/lambda.cfg')

time: 1.2 ms


# Breakdown Demo

In [6]:
client = CalculatorClient(aws_info)

time: 735 µs


In [7]:
responses = client.get_breakdown(uids, pids[:2])

print '{} responses returned'.format(len(responses))

2000 responses returned
time: 20.2 s


In [8]:
responses = client.get_breakdown(uids, pids[:2], use_s3_for_claims=False, max_calculated_uids=100)

print '{} responses returned'.format(len(responses))

2000 responses returned
time: 18 s


# Scenario Similar to Commercial

In [9]:
responses = client.get_oop(uids[:300], pids, use_s3_for_claims=False)

print '{} responses returned'.format(len(responses))

80400 responses returned
time: 19.2 s


# S3 versus DynamoDB as Claims Storage

In [10]:
# Use S3:
claims_client = ClaimsClient(aws_info, 
                             s3_bucket=configs.claims_bucket,
                             s3_path=configs.claims_path)
people = claims_client.get(uids[:300])

print 'claims for {} people read'.format(len(people))

claims for 300 people read
time: 49.6 s


In [11]:
person = people[0]
print person.keys()
{
    'uid': person['uid'],
    'medical_claims': person['medical_claims'][:5]
}

['medical_claims', 'uid']


{'medical_claims': [{'admitted': '2014-04-03',
   'benefit_category': 16,
   'cost': 148.0,
   'discharged': '2014-04-03',
   'length_of_stay': 1},
  {'admitted': '2014-05-02',
   'benefit_category': 16,
   'cost': 74.55,
   'discharged': '2014-05-02',
   'length_of_stay': 1},
  {'admitted': '2014-05-05',
   'benefit_category': 16,
   'cost': 104.39,
   'discharged': '2014-05-05',
   'length_of_stay': 1},
  {'admitted': '2014-05-13',
   'benefit_category': 16,
   'cost': 210.12,
   'discharged': '2014-05-13',
   'length_of_stay': 1},
  {'admitted': '2014-05-19',
   'benefit_category': 11,
   'cost': 442.2,
   'discharged': '2014-05-19',
   'length_of_stay': 1}],
 'uid': '1302895801'}

time: 14.9 ms


In [12]:
# Use DynamoDB:
claims_client = ClaimsClient(aws_info, 
                             table_name=configs.claims_table)
people = claims_client.get(uids[:300])

print 'claims for {} people read'.format(len(people))

claims for 300 people read
time: 3.16 s
